An __Exploratory Data Analysis__ has been conducted on a Mexican dataset covering the early stages of the COVID-19 pandemic.

Source: https://www.kaggle.com/datasets/meirnizri/covid19-dataset.

__The objective__ of the analysis is to identify factors influencing the progression of the COVID-19 disease and, consequently, enhance prognostication.

The dataset includes information on various factors such as age, gender, pregnancy, comorbid diseases, obesity, medication, and the history of treatment (whether a patient was admitted to the hospital, intubated, etc.).

__Conclusions:__

1. The highest number of cases is reported in medical units ‘4’ and ‘12’. These units may be particularly susceptible to the influx of COVID-19 patients. The provided information will aid, among other things, in:
    - Prioritizing the allocation of equipment and medical personnel resources.
    - Planning interventions and allocating preventive measures to minimize future waves of infections.
    - Identifying risk factors in these units.

2. The development of pneumonia significantly increases the mortality rate, underscoring the critical importance of effective pneumonia treatment to enhance the chances of survival.

3. Nearly 60% of individuals considered 'healthy' (without any chronic diseases listed in the dataset) have contracted pneumonia. This suggests that being healthy does not necessarily reduce the risk of a severe course of COVID-19.

4. Factors such as obesity and smoking do not appear to predispose patients to a more severe illness.


As the analyzed dataset only spans the initial stages of the COVID-19 pandemic, it is recommended to perform additional analysis using more recent data.

In [ ]:
import datetime
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#importing data and replacing fake dates with NaT 
df = pd.read_csv('Covid Data.csv')
df.head(20)

df['DATE_DIED'] = df['DATE_DIED'].replace('9999-99-99', pd.NaT)



In [ ]:
#As '97' and '99' values are in fact missing data; rows containing it will be filtered out 
temp_list = [97,98,99]
df_analysis = df[(~df["ICU"].isin(temp_list))&(~df["TOBACCO"].isin(temp_list))&(~df["RENAL_CHRONIC"].isin(temp_list))&(~df["OBESITY"].isin(temp_list))& (~df["CARDIOVASCULAR"].isin(temp_list))& (~df["OTHER_DISEASE"].isin(temp_list))& (~df["HIPERTENSION"].isin(temp_list))& (~df["INTUBED"].isin(temp_list)) & (~df["PNEUMONIA"].isin(temp_list)) & (~df["PREGNANT"].isin(temp_list)) & (~df["DIABETES"].isin(temp_list))& (~df["INTUBED"].isin(temp_list)) & (~df["PNEUMONIA"].isin(temp_list)) & (~df["COPD"].isin(temp_list)) & (~df["ASTHMA"].isin(temp_list)) & (~df["INMSUPR"].isin(temp_list))]

In [ ]:
#creating a new variable indicating whether a patient has or has not been diagnosed with Covid
df_analysis["DIAGNOSED_COVID"] = np.where((df_analysis["CLASIFFICATION_FINAL"] > 3), 2, 1)

In [ ]:
#checking the number of duplicates
df.duplicated().sum()

In [ ]:
#removing duplicates
df_analysis2 = df_analysis.drop_duplicates()

In [ ]:
df_analysis2

In [ ]:
df_analysis2.count()

In [ ]:
a = df.head()["DATE_DIED"]
print(a)
print(type(a))

In [ ]:
age = df_analysis2["AGE"]
age.info()

age.hasnans #checking missing data


In [ ]:
age.quantile(.1), age.quantile(.25), age.quantile(.5), age.quantile(.75), age.quantile(.99)

In [ ]:
quartiles = pd.DataFrame({'q25' : [45], 'q50' : [59], 'q75': [71]})

quartiles.style \
    .relabel_index(["age"])


In [ ]:
avg_age = df_analysis2["AGE"].mean()
avg_age

In [ ]:
df_analysis2['AGE'].min(), df_analysis2['AGE'].max()

In [ ]:
#creating histograms for each variable to see 
df_analysis2.hist(figsize=(20,20));

In [ ]:
plt.hist(data = df_analysis2, x="AGE", color="Pink", ec='Red')
plt.xlabel('Age')
plt.ylabel('# The number of people infected');

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
sns.set_style("ticks")
ax = sns.boxplot(x="AGE", y="PATIENT_TYPE", data=df_analysis2, orient='h', color = 'blue')


In [ ]:
#creating a population of patients who died
df_died = df_analysis2.query('~DATE_DIED.isnull()')
df_died.count()

In [ ]:
#calculating the death ratio
df_died["USMER"].count()/df_analysis2["USMER"].count() 

In [ ]:
#comparing the average age of a died person withe the average age of a patient
df_died["AGE"].mean(), df_analysis2["AGE"].mean()  

In [ ]:
# After filtering out missing data and removing duplicates there are only women left in the dataset (SEX=1)
df_analysis2.query("SEX ==1").count() 


In [ ]:
df_died["AGE"].min(), df_died["AGE"].max()

In [ ]:
#78 pregnant women died 
df_died.query("PREGNANT == 1")

In [ ]:
#The average age of a pregnant woman was 31
df_died.query("PREGNANT == 1")["AGE"].mean() 

In [ ]:
#The average age of a woman who was not pregnant
df_died.query("PREGNANT != 1")["AGE"].mean() 

In [ ]:
#calculating pneumonia ratio
df_analysis2.query("PNEUMONIA ==1")["USMER"].count()/df_analysis2["USMER"].count() 

In [ ]:
#calculating pneumonia ratio in the deceased population
df_died.query("PNEUMONIA ==1")["USMER"].count()/df_died["USMER"].count()

In [ ]:
#pneumonia ratio

pneumonia = df_analysis2.query('PNEUMONIA == 1')['PNEUMONIA'].count()
not_pneumonia = df_analysis2.query('PNEUMONIA == 2 or PNEUMONIA == 99')['PNEUMONIA'].count()

pneumonia_proportion = pneumonia / df_analysis2['PNEUMONIA'].count()
not_pneumonia_proportion = not_pneumonia / df_analysis2['PNEUMONIA'].count()

#the death ratio in case of pneumonia infection

died_pneumonia = df_died.query('PNEUMONIA == 1')['PNEUMONIA'].count()
died_not_pneumonia = df_died.query('PNEUMONIA == 2 or PNEUMONIA == 99')['PNEUMONIA'].count()

died_pneumonia_proportion = died_pneumonia / df_died['PNEUMONIA'].count()
died_not_pneumonia_proportion = died_not_pneumonia / df_died['PNEUMONIA'].count()


died_proportion = df_died["USMER"].count()/df_analysis2["USMER"].count()

not_died_proportion = 1 - died_proportion



In [ ]:
died_pregnant = df_died.query("PREGNANT == 1").count()
pregnant = df_analysis2.query("PREGNANT == 1").count()

died_pregnant_proportion = died_pregnant / pregnant

In [ ]:
# First subplot
plt.subplot(1, 2, 1)

# labels_population = ['Died', 'Survived']
sizes_population = [died_proportion, not_died_proportion]
colors_population = ['orange', 'purple']
explode_population = (0, 0.1)

plt.pie(sizes_population, explode=explode_population, colors=colors_population,
        autopct='%1.1f%%', shadow=True, startangle=0)
plt.axis('equal')
plt.title("The death ratio in population")



# Second subplot
plt.subplot(1, 2, 2)

# labels_pneumonia = ['Died - pneumonia', 'Survived - pneumonia']
sizes_pneumonia = [died_pneumonia_proportion, died_not_pneumonia_proportion]
colors_pneumonia = ['orange', 'purple']
explode_pneumonia = (0, 0.1)

plt.pie(sizes_pneumonia, explode=explode_pneumonia, colors=colors_pneumonia,
        autopct='%1.1f%%', shadow=True, startangle=-50)
plt.axis('equal')
plt.title("The death ratio among patients with pneumonia", loc='left')


legend_labels = ['Died', 'Survived']
plt.legend(legend_labels, title="Legend", loc='lower right', bbox_to_anchor=(1.5, 0.5))


# Show the plot
plt.show()

In [ ]:
#analyzing diabetics in population
df_analysis2.query("DIABETES ==1")["USMER"].count()/df_analysis2["USMER"].count() 

In [ ]:
#the proportion of diabetics in the population of the deceased
df_died.query("DIABETES ==1")["USMER"].count()/df_died["USMER"].count() 

In [ ]:
# Chronic Obstructive Pulmonary Disease
df_analysis2.query("COPD ==1")["USMER"].count()/df_analysis2["USMER"].count()

In [ ]:
#The death ratio among patients with chronic obstructive pulmonary disease is 42%, compared to the overall mortality rate of 45% (calculated above)
df_died.query("COPD ==1").count()/df_analysis2.query("COPD ==1").count() 

In [ ]:
#creating a population of so called healthy (without any concomitant disease listed in the dataset)
df_healthy = df_analysis2.query("DIABETES == 2 & COPD ==2 & ASTHMA ==2 & INMSUPR == 2 & HIPERTENSION ==2 & OTHER_DISEASE ==2 & CARDIOVASCULAR ==2 & RENAL_CHRONIC==2") 

In [ ]:
#Nearly 60% of individuals, so-called "healthy" (without comorbidities), developed pneumonia. Interestingly, the pneumonia rate in the entire population is 62% (calculated above).

df_healthy.query("PNEUMONIA ==1").count()/df_healthy["USMER"].count() 



In [ ]:
#creating the population of people who suffer from at least one concomitant disease (from the list)
df_sick = df_analysis2.query("DIABETES == 1 or COPD ==1 or ASTHMA ==1 or INMSUPR == 1 or HIPERTENSION ==1 or OTHER_DISEASE ==1 or CARDIOVASCULAR ==1 or RENAL_CHRONIC==1")

In [ ]:
df_sick["USMER"].count()/df_analysis2["USMER"].count() #Individuals with chronic illnesses, accounting for at least one disease, constitute 70% of the population.

In [ ]:
df_sick.query("INTUBED ==1").count()/df_sick.count()

In [ ]:
df_healthy.query("INTUBED ==1").count()/df_healthy.count()

In [ ]:
df_healthy.query("PNEUMONIA ==1").count()/df_healthy.count()

In [ ]:
df_analysis2.query("PNEUMONIA ==1").count()/df_analysis2.count()

In [ ]:
#The ratio of chronically ill individuals among the deceased to the total number of deceased is only slightly higher than the rate of sick individuals in the population.
df_died.query("DIABETES == 1 or COPD ==1 or ASTHMA ==1 or INMSUPR == 1 or HIPERTENSION ==1 or OTHER_DISEASE ==1 or CARDIOVASCULAR ==1 or RENAL_CHRONIC==1")["USMER"].count()/df_died["USMER"].count() 

In [ ]:
# An analysis of the obese patients
df_analysis2.query("OBESITY ==1").count()/df_analysis2["USMER"].count()


In [ ]:
#In the population, approximately 27% of individuals are obese. The mortality rate among obese individuals is 26%, indicating that obesity does not increase the risk of mortality.
df_died.query("OBESITY ==1").count()/df_died["USMER"].count() 

In [ ]:
# Smokers
df_analysis2.query('TOBACCO ==1')["USMER"].count()/df_analysis2["USMER"].count() 

In [ ]:
#Among the deceased population, smokers constitute 4%, which is a lower percentage of smokers among the deceased than in the entire population (6%).
df_died.query("TOBACCO ==1").count()/df_died["USMER"].count() 

In [ ]:
# The pneumonia rate among obese patients is only slightly higher than the pneumonia  rate in the general population (62%).
df_analysis2.query('PNEUMONIA ==1 and OBESITY ==1')["USMER"].count()/df_analysis2.query("OBESITY ==1")["USMER"].count() 

In [ ]:
df_analysis2.query('PNEUMONIA ==1')["USMER"].count()/df_analysis2["USMER"].count() 

In [ ]:
# Hospitalizowany czy odesłany do domu?
# 1 - returned home
# 2 - hospitalization

df_died.query("PATIENT_TYPE ==1")["USMER"].count()/df_died["USMER"].count() #There were no patients who were returned home and died afterwards.

In [ ]:
df_died.query("PATIENT_TYPE ==2")["USMER"].count()/df_died["USMER"].count() #All the deceased were hospitalised.

In [ ]:
# The death ratio among hospitalised patients is 45%.

df_died.query("PATIENT_TYPE ==2")["USMER"].count()/df_analysis2.query("PATIENT_TYPE ==2")["USMER"].count() 

In [ ]:
#CLASSIFICATION_FINAL {1,2,3} means that a patient was diagnosed with COVID, {4,5,6,7} means that a patient was not diagnosed or the result was ambiguous.

df_died.query("CLASIFFICATION_FINAL == 1 or CLASIFFICATION_FINAL == 2 or CLASIFFICATION_FINAL == 3")["USMER"].count()/df_analysis2.query("CLASIFFICATION_FINAL == 1 or CLASIFFICATION_FINAL == 2 or CLASIFFICATION_FINAL == 3")["USMER"].count()

#The death ratio (57%) among patients diagnosed with Covid is higher when compared to population (45%).


In [ ]:
#ICU - Intensive Care Unit

df_died.query("ICU ==1")["USMER"].count()/df_died["USMER"].count() #11% of all deceased individuals were in the ICU.

In [ ]:
df_analysis2["AGE"].corr(df_analysis["PNEUMONIA"], method='pearson', min_periods=None) #no significant correlation between age and the fact of being diagnosed with pneumonia


In [ ]:
df_correlation = df_analysis2.drop(columns = ["SEX", "PATIENT_TYPE", "DATE_DIED"])


In [ ]:
#correlation matrix
corr = round(df_correlation[df_correlation.columns].corr(),2)
plt.rcParams['figure.figsize']=(15,10)
%time ax = sns.heatmap(corr, vmax=1., vmin=-1., annot=True, linewidths=.8, cmap="Oranges")

# correcting matplotlib issue with cutting edges
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5);




In [ ]:
# Defining age buckets and lables for a heatmap
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 130]

labels = ['(0;10]', '(11;20]', '(21;30]', '(31;40]', '(41;50]', '(51;60]', '(61;70]', '(71;80]', '(81;90]', '(91;100]','(101+)']

df_analysis2['age_group'] = pd.cut(df_analysis2['AGE'], bins=bins, labels=labels, right=True)


In [ ]:

# Zamiana daty śmierci na 1, a brakujące dane (NaN) na 0 w nowej kolumnie "DEAD"
df_analysis2s['DEAD'] = df_analysis2['DATE_DIED'].notnull().astype(int)

# Tworzenie DataFrame'u przystosowanego do heatmapy
df_analysis2_heatmap = df_analysis2.pivot_table(index='age_group', columns='DIAGNOSED_COVID', values='DEAD', aggfunc='sum', fill_value=0)

# Tworzenie heatmapy przy użyciu Seaborn
plt.figure(figsize=(5, 5))
sns.heatmap(df_analysis2_heatmap, cmap="BuPu")

# Odwracanie osi y
plt.gca().invert_yaxis()

plt.title("Heatmap of the deceased")
plt.xlabel("Diagnosed Covid ")
plt.ylabel("Age")

# Wyświetlanie wykresu
plt.show()